In [5]:
with open('/home/blu-bridge004/Desktop/GPT-Clone-main/GPT-Clone/neural netwoks/input files/gpt/grimms_fairytales.txt','r',encoding='utf-8') as f:
    text=f.read()
len(text)

540138

OrderedDict([('token_embedding_table.weight',
              tensor([[ 0.0083,  0.0347, -0.1138,  ..., -0.0621,  0.0049, -0.0010],
                      [ 0.0471,  0.0312, -0.0054,  ..., -0.0155, -0.0076, -0.0006],
                      [-0.0182,  0.0149,  0.0038,  ...,  0.0404, -0.0443, -0.0286],
                      ...,
                      [ 0.0856,  0.0194,  0.0971,  ...,  0.0173,  0.0097, -0.0219],
                      [ 0.0301, -0.0640,  0.0687,  ..., -0.0157, -0.0185,  0.0234],
                      [ 0.0387, -0.0694, -0.0608,  ..., -0.0286,  0.0141,  0.0878]],
                     device='cuda:0')),
             ('position_embedding_table.weight',
              tensor([[-0.0479,  0.2805, -0.2777,  ...,  0.1559,  0.0219,  0.0327],
                      [-0.0521,  0.0520,  0.0909,  ...,  0.1215,  0.0033,  0.0075],
                      [ 0.0313,  0.0117, -0.0195,  ...,  0.0519,  0.0169,  0.0434],
                      ...,
                      [-0.0040,  0.0181, -0.0116,  ...

In [6]:
print(text[:1000])

The Project Gutenberg eBook of Grimms’ Fairy Tales, by Jacob Grimm and Wilhelm Grimm

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.

Title: Grimms’ Fairy Tales

Author: Jacob Grimm and Wilhelm Grimm

Translators: Edgar Taylor and Marian Edwardes

Release Date: April, 2001 [eBook #2591]
[Most recently updated: June 28, 2021]

Language: English

Character set encoding: UTF-8

Produced by: Emma Dudding, John Bickers, Dagny and David Widger

*** START OF THE PROJECT GUTENBERG EBOOK GRIMMS’ FAIRY TALES ***




Grimms’ Fairy Tales

By Jacob Grimm and Wilhelm Grimm



PREPARER’S NOTE

     The text is 


In [8]:
chars=sorted(list(set(text)))
print(''.join(chars))
vocab_size=len(chars)
print(vocab_size)


 !"#$%'()*,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyzä‘’“”﻿
89


In [161]:
stoi={ch:i for i,ch in enumerate(chars)}
itos={i:ch for i,ch in enumerate(chars)}
encode=lambda s: [stoi[c] for c in s] # encoder : take a string and output a list of integers
decode=lambda l: ''.join([itos[i] for i in l]) # decoder : takes a list and output a string
print(encode("hii there"))
print(decode(encode("hii there")))

[46, 47, 47, 1, 58, 46, 43, 56, 43]
hii there


In [162]:
import tiktoken
enc=tiktoken.encoding_for_model("gpt-2")
print(enc.n_vocab) #for gpt 2 the token size ranges from 0 to 50257
enc.encode("hii there")
enc.decode([71,4178,612])

50257


'hii there'

In [163]:
import torch
data=torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

In [164]:
n=int(0.9 * len(data))
train_data=data[:n]
val_data=data[n:]


In [165]:
block_size=8
train_data[:block_size+1]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])

In [166]:
x=train_data[:block_size]
y=train_data[1:block_size+1]
for t in range(block_size):
    context=x[:t+1]
    target=y[t]
    print(f"when input is {context} the target :{target}")


when input is tensor([18]) the target :47
when input is tensor([18, 47]) the target :56
when input is tensor([18, 47, 56]) the target :57
when input is tensor([18, 47, 56, 57]) the target :58
when input is tensor([18, 47, 56, 57, 58]) the target :1
when input is tensor([18, 47, 56, 57, 58,  1]) the target :15
when input is tensor([18, 47, 56, 57, 58,  1, 15]) the target :47
when input is tensor([18, 47, 56, 57, 58,  1, 15, 47]) the target :58


In [167]:
torch.manual_seed(1337)
batch_size=4 #how many independent sequences will be process in parallel?
block_size=8 # context length
def get_batch(split):
    data=train_data if split=="train" else val_data
    ix=torch.randint(len(data)-block_size,(batch_size,)) #it randomly takes 4 indices
    x=torch.stack([data[i:i+block_size] for i in ix]) # it fetches the value at that index
    y=torch.stack([data[i+1:i+block_size+1] for i in ix]) #offset by 1 ie targets
    return x,y
xb,yb=get_batch('train')
print("inputs")
print(xb.shape)
print(xb)
print("targets")
print(yb.shape)
print(yb)
print("-------------------")
for b in range(batch_size):
    for t in range(block_size):
        context=xb[b,:t+1]
        target=yb[b,t]
        print(f"when input is {context.tolist()} the target:{target}")

inputs
torch.Size([4, 8])
tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])
targets
torch.Size([4, 8])
tensor([[43, 58,  5, 57,  1, 46, 43, 39],
        [53, 56,  1, 58, 46, 39, 58,  1],
        [58,  1, 58, 46, 39, 58,  1, 46],
        [17, 27, 10,  0, 21,  1, 54, 39]])
-------------------
when input is [24] the target:43
when input is [24, 43] the target:58
when input is [24, 43, 58] the target:5
when input is [24, 43, 58, 5] the target:57
when input is [24, 43, 58, 5, 57] the target:1
when input is [24, 43, 58, 5, 57, 1] the target:46
when input is [24, 43, 58, 5, 57, 1, 46] the target:43
when input is [24, 43, 58, 5, 57, 1, 46, 43] the target:39
when input is [44] the target:53
when input is [44, 53] the target:56
when input is [44, 53, 56] the target:1
when input is [44, 53, 56, 1] the target:58
when input is [44, 53, 56, 1, 58] the target:46
when input is [44, 53

In [168]:
print(xb)

tensor([[24, 43, 58,  5, 57,  1, 46, 43],
        [44, 53, 56,  1, 58, 46, 39, 58],
        [52, 58,  1, 58, 46, 39, 58,  1],
        [25, 17, 27, 10,  0, 21,  1, 54]])


In [169]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)
class BigramlanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table=nn.Embedding(vocab_size,vocab_size) #65,65 each has vector of weights
    def forward(self,idx,targets=None):
        logits=self.token_embedding_table(idx) #from 4,8 to 4,8,65
        if targets==None:
            loss=None
        else:
            B,T,C =logits.shape
            logits=logits.view(B*T,C) #cross entropy changes the dimension in multidimension so we changed the dim
            targets=targets.view(B*T)
            loss=F.cross_entropy(logits,targets)
        return logits,loss
    def generate(self,idx,max_new_tokens):
        #idx is (B,T) array of indices in the current context #initally (1,1)
        for _ in range(max_new_tokens):
           logits,loss=self(idx)
           # focus only on the last time step
           logits=logits[:,-1,:] #becomes(B,C) we use -1 beacuse we need the last timestep to predict the next token
           probs = F.softmax(logits, dim=-1) # (B, C) 
           # sample from the distribution
           idx_next = torch.multinomial(probs, num_samples=1) # (B, 1) it normalizes all the 65 values and based on probs it give single value
           # append sampled index to the running sequence
           idx = torch.cat((idx, idx_next), dim=1) # (B, T+1) inceases the timestep 
        return idx #now it becomes(4,8+100)
             


m=BigramlanguageModel(vocab_size)
logits,loss=m(xb,yb)
print(logits.shape)
print(loss)
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 65])
tensor(4.8786, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [170]:
optimizer=torch.optim.AdamW(m.parameters(),lr=1e-3)

In [171]:
batch_size=32
for steps in range(10000):
    xb,yb=get_batch("train")
    logits,loss=m(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

2.5727508068084717


In [172]:
print(decode(m.generate(idx=torch.zeros((1,1),dtype=torch.long), max_new_tokens=300)[0].tolist()))


Iyoteng h hasbe pave pirance
Rie hicomyonthar's
Plinseard ith henoure wounonthioneir thondy, y heltieiengerofo'dsssit ey
KIN d pe wither vouprrouthercc.
hathe; d!
My hind tt hinig t ouchos tes; st yo hind wotte grotonear 'so it t jod weancotha:
h hay.JUCle n prids, r loncave w hollular s O:
HIs; ht 


In [173]:
torch.manual_seed(1337)
B,T,C=4,8,32
x=torch.randn(B,T,C) 
x.shape

torch.Size([4, 8, 32])

In [174]:
xbow=torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev=x[b,:t+1] # (8,2) from(0,2) to (7,2) to  and it takes average and stores in xbow
        xbow[b,t]=torch.mean(xprev,0) #4,8,2 it takes averaGES on 2 valoes of each timestamp

In [175]:
wei=torch.tril(torch.ones(T,T)) # it change the upper right triangle as 0
wei=wei/wei.sum(1,keepdim=True) #normalizing each row
xbow2=wei @ x #1,8,8--- 4,8,2====>>
torch.allclose(xbow,xbow2)

False

In [180]:
#lets see a single head perform self attention
head_size=16
#each token has two vectors key and query
key=nn.Linear(C,head_size,bias=False) #what do i have #B,T,16
query=nn.Linear(C,head_size,bias=False) #what am i looking for?   #B,T,16
value=nn.Linear(C,head_size,bias=False) # "What information do I pass forward?
k=key(x)
q=query(x)
wei=q @ k.transpose(-2,-1) #(B,T,16) @ (B,16,T) ----> (B,T,T)
tril=torch.tril(torch.ones(T,T))
#wei=torch.zeros(T,T) here the wei is uniformly ditributed along the rows and there is no connection between the tokens
wei=wei.masked_fill(tril==0,float('-inf')) #to avoid future communication we made upper as 0 using mask
wei=F.softmax(wei,dim=-1) #used to normalize the wei in a nice distribution
v=value(x) #v=4,8,16  
#out=wei @ x #actually we dont aggregate the exact value of token
out=wei @ v #8,8 @ 4,8,16--->4,8,16
out.shape


torch.Size([4, 8, 16])

In [2]:
import tiktoken
enc = tiktoken.get_encoding("gpt2")
tokens = enc.encode("o200k")
print(tokens, len(tokens))


[78, 2167, 74] 3


In [1]:


import requests

url = "https://www.gutenberg.org/files/2591/2591-0.txt"
response = requests.get(url)

with open("grimms_fairytales.txt", "w", encoding="utf-8") as f:
    f.write(response.text)
